### Load Data 

In [2]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import requests

In [13]:
url = 'http://world.openfoodfacts.org/data/en.openfoodfacts.org.products.csv'

In [3]:
food = pd.read_csv('en.openfoodfacts.org.products.csv', sep='\t', error_bad_lines=False)

Skipping line 78597: expected 159 fields, saw 161

/home/jonathan/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0,3,5,27,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
food = pd.read_csv(url, sep='\t', error_bad_lines=False)

/home/jonathan/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0,3,5,27,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Select Variables 

In [16]:
col_select = ['url', 'code',
              'product_name', 'stores', 'countries_en',
              'traces_en',
              'serving_size',
              'energy_100g', 
              'carbohydrates_100g', 'sugars_100g', 'sodium_100g', 
              'fat_100g', 'saturated-fat_100g',
              'fiber_100g', 'proteins_100g', 
              'alcohol_100g', 'caffeine_100g']

In [20]:
food_slim = food.loc[:,col_select]
food_slim.shape

(87965, 17)

### Remove Null Lines

In [21]:
# Null Product Name
food_set = food_slim.copy()
# Remove Null Name Rows
food_set = food_set.loc[~food_set.product_name.isnull()]
# Remove Null Country Rows
food_set = food_set.loc[~food_set.countries_en.isnull()]
# Remove Null Code Rows
food_set = food_set.loc[~food_set.code.isnull()]
food_set.shape

(77677, 17)

In [22]:
def null_check(col_list, df):
    out = {}
    for col in col_list:
        c = Counter(df[col].isnull())
        out[col] = {'Null': c[True], 'Real': c[False]}
    return pd.DataFrame(out).T

In [23]:
null_df = null_check(col_select, food_set).reset_index()
null_df

,index,Null,Real
0,alcohol_100g,74258,3419
1,caffeine_100g,77636,41
2,carbohydrates_100g,32317,45360
3,code,0,77677
4,countries_en,0,77677
5,energy_100g,31875,45802
6,fat_100g,31916,45761
7,fiber_100g,50420,27257
8,product_name,0,77677
9,proteins_100g,32434,45243


### Remove Products with Complex Names 

In [24]:
clean_name = [',' not in v for v in food_set['product_name'].values]
food_clean_name = food_set.loc[clean_name]

In [25]:
def comma_check(col_list, df):
    out = {}
    for col in col_list:
        out[col] = sum([',' in str(v) for v in df[col].values])
    return out

In [26]:
clean_name_summary = comma_check(col_select, food_clean_name)
x = pd.DataFrame({'CommaRow': clean_name_summary}).sort_values('CommaRow', ascending=False)
x.loc[x.CommaRow>0,]

,CommaRow
traces_en,11750
stores,2158
serving_size,1825
countries_en,1769


In [27]:
food_clean_name.reset_index(drop=True, inplace=True)
print food_clean_name.shape
food_clean_name.head()

(74901, 17)


,url,code,product_name,stores,countries_en,traces_en,serving_size,energy_100g,carbohydrates_100g,sugars_100g,sodium_100g,fat_100g,saturated-fat_100g,fiber_100g,proteins_100g,alcohol_100g,caffeine_100g
0,http://world-en.openfoodfacts.org/product/0000...,0000000024600,Filet de bœuf,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://world-en.openfoodfacts.org/product/0000...,0000000036252,Lion Peanut x2,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://world-en.openfoodfacts.org/product/0000...,0000000039259,Twix x2,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://world-en.openfoodfacts.org/product/0000...,0000000039529,Pack de 2 Twix,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://world-en.openfoodfacts.org/product/0000...,0000000056434,diet lemonade by Sainsbury's,NaN,United Kingdom,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Check Countries Count 

In [28]:
country = food_clean_name.countries_en.apply(lambda r: r.lower().split(','))

In [29]:
country_list_flat = [c for subc in country for c in subc]

In [30]:
country_count_df = pd.DataFrame({'Count': Counter(country_list_flat)})\
                            .reset_index()\
                            .sort_values('Count', ascending=False)

### Filter countries that appears more than 300 times

In [31]:
country_cut = country_count_df.loc[country_count_df.Count > 300]
country_cut

,index,Count
53,france,50677
58,germany,4721
136,spain,3818
153,united kingdom,3380
154,united states,2311
15,belgium,2252
140,switzerland,2086
9,australia,1517
74,italy,589
117,portugal,583


In [32]:
country_final = country_cut['index'].tolist()
country_final = [c for c in country_final]
# country_final.remove('fr:Europe')

In [33]:
print len(country_final)
print country_final

14
['france', 'germany', 'spain', 'united kingdom', 'united states', 'belgium', 'switzerland', 'australia', 'italy', 'portugal', 'canada', 'denmark', 'netherlands', 'brazil']


### Country flags for each product

In [34]:
country_count_df = pd.DataFrame(np.zeros([food_clean_name.shape[0], # Product Count
                                          len(country_final)],      # Country Count
                                         dtype='int'))
country_count_df.columns = country_final
country_count_df.head()

,france,germany,spain,united kingdom,united states,belgium,switzerland,australia,italy,portugal,canada,denmark,netherlands,brazil
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
for i in country_count_df.index:
    for c in country[i]:
        if c in country_count_df.columns:
            country_count_df.loc[i, c] += 1

In [36]:
country_count_df.sum() # Matches country_cut

france            50677
germany            4721
spain              3818
united kingdom     3380
united states      2311
belgium            2252
switzerland        2086
australia          1517
italy               589
portugal            583
canada              525
denmark             356
netherlands         325
brazil              307
dtype: int64

In [37]:
food_final  = pd.concat([food_clean_name, country_count_df], axis = 1).drop('countries_en', axis = 1)

In [38]:
food_final.shape

(74901, 30)

In [39]:
food_final.head()

,url,code,product_name,stores,traces_en,serving_size,energy_100g,carbohydrates_100g,sugars_100g,sodium_100g,...,united states,belgium,switzerland,australia,italy,portugal,canada,denmark,netherlands,brazil
0,http://world-en.openfoodfacts.org/product/0000...,0000000024600,Filet de bœuf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,http://world-en.openfoodfacts.org/product/0000...,0000000036252,Lion Peanut x2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,http://world-en.openfoodfacts.org/product/0000...,0000000039259,Twix x2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,http://world-en.openfoodfacts.org/product/0000...,0000000039529,Pack de 2 Twix,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,http://world-en.openfoodfacts.org/product/0000...,0000000056434,diet lemonade by Sainsbury's,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [40]:
food_final.shape

(74901, 30)

In [42]:
# Remove Extreme values
food_final = food_final.loc[(food_final.sodium_100g != food_final.sodium_100g.max())
                            & (food_final.energy_100g != food_final.energy_100g.max())]
food_final.shape

(74899, 30)

### Write Files

In [43]:
!pwd

/media/sf_D_DRIVE/BootCamp/Projects/Shiny/FoodExplorer/Data


In [45]:
# Food Data
food_final.columns = [v.capitalize() for v in food_final.columns.values]
food_final.to_csv('Food_Cleaned.csv', index = False)

In [46]:
# Country Summary
country_cut['index'] = country_cut['index'].apply(lambda s: s.capitalize())
country_cut.to_csv('Country_Stat.csv', index = False)